In [74]:
import pandas as pd

In [75]:
df = pd.read_csv('dataset\Raw_data_1Hr_2015_2024_Mandir_Marg_Delhi_DPCC.csv')

In [76]:
df.head()

,Timestamp,PM2.5,PM10,NO,NO2,NH3,SO2,CO,Ozone,AT,RH,WS,WD,SR,BP
0,01-01-2015 0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01-01-2015 1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01-01-2015 2.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01-01-2015 3.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01-01-2015 4.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [77]:
df.columns

Index(['Timestamp', 'PM2.5', 'PM10', 'NO', 'NO2', 'NH3', 'SO2', 'CO', 'Ozone',
       'AT', 'RH', 'WS', 'WD', 'SR', 'BP'],
      dtype='object')

In [78]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87672 entries, 0 to 87671
Data columns (total 15 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Timestamp  87672 non-null  object 
 1   PM2.5      76973 non-null  float64
 2   PM10       77234 non-null  float64
 3   NO         75595 non-null  float64
 4   NO2        76555 non-null  float64
 5   NH3        75934 non-null  float64
 6   SO2        74546 non-null  float64
 7   CO         75144 non-null  float64
 8   Ozone      75821 non-null  float64
 9   AT         77414 non-null  float64
 10  RH         77508 non-null  float64
 11  WS         78462 non-null  float64
 12  WD         77639 non-null  float64
 13  SR         78021 non-null  float64
 14  BP         78037 non-null  float64
dtypes: float64(14), object(1)
memory usage: 10.0+ MB


In [79]:
df.describe()

,PM2.5,PM10,NO,NO2,NH3,SO2,CO,Ozone,AT,RH,WS,WD,SR,BP
count,76973.000000,77234.000000,75595.000000,76555.000000,75934.000000,74546.000000,75144.00000,75821.000000,77414.000000,77508.000000,78462.000000,77639.000000,78021.000000,78037.000000
mean,100.399404,183.924109,30.276559,56.578669,27.399659,11.598617,1.22733,26.411646,24.017836,62.234490,1.132568,220.520490,94.616611,906.180131
std,92.457759,122.202545,48.371986,31.514481,17.446663,9.320583,1.24745,31.622872,9.033464,22.377142,0.868777,58.540136,130.222565,112.910540
min,0.270000,0.170000,0.060000,0.240000,0.100000,0.030000,0.00000,0.020000,0.100000,1.030000,0.080000,0.030000,0.000000,732.350000
25%,37.500000,95.000000,3.770000,34.880000,15.580000,6.133333,0.64000,7.450000,17.300000,44.580000,0.320000,180.565000,5.330000,737.000000
50%,70.670000,157.000000,11.160000,50.480000,22.280000,9.720000,0.92000,14.300000,25.120000,65.500000,0.920000,237.125000,15.425000,977.200000
75%,134.750000,243.500000,35.190000,71.865000,34.400000,14.280000,1.42000,32.560000,30.030000,81.900000,1.720000,264.550000,157.020000,985.750000
max,996.000000,992.000000,498.400000,434.430000,282.700000,190.100000,49.62000,368.720000,49.800000,100.000000,17.990000,349.320000,621.580000,1000.225000


In [80]:
df.isna().sum()

Timestamp        0
PM2.5        10699
PM10         10438
NO           12077
NO2          11117
NH3          11738
SO2          13126
CO           12528
Ozone        11851
AT           10258
RH           10164
WS            9210
WD           10033
SR            9651
BP            9635
dtype: int64

In [81]:
df['ds'] = pd.to_datetime(df['Timestamp'], format="%d-%m-%Y %H.%M") #datetime setup

In [82]:
df['y'] = df['PM2.5']

In [83]:
df = df.dropna(subset=['y']).reset_index(drop=True) #drop rows where PM2.5 is missing

In [84]:
#decide features
base_features = ['PM10', 'NO', 'NO2', 'NH3', 'SO2', 'CO', 'Ozone', 'AT', 'RH', 'WS', 'WD', 'SR', 'BP']

### Now Training Prophet Model(only for time patterns)

In [85]:
from prophet import Prophet
prophet_df = df[['ds', 'y']]
prophet = Prophet(
    daily_seasonality=True,
    weekly_seasonality=True,
    yearly_seasonality=True
)

prophet.fit(prophet_df)


19:11:16 - cmdstanpy - INFO - Chain [1] start processing
19:12:21 - cmdstanpy - INFO - Chain [1] done processing


In [86]:
#now generate Prophet features
prophet_out = prophet.predict(prophet_df)

df['trend'] = prophet_out['trend']
df['daily'] = prophet_out['daily']
df['weekly'] = prophet_out['weekly']
df['yearly'] = prophet_out['yearly']
df['prophet_yhat'] = prophet_out['yhat']


In [87]:
df.head()

,Timestamp,PM2.5,PM10,NO,NO2,NH3,SO2,CO,Ozone,AT,...,WD,SR,BP,ds,y,trend,daily,weekly,yearly,prophet_yhat
0,09-04-2015 15.00,38.00,61.00,NaN,NaN,NaN,8.17,1.08,112.91,30.22,...,233.33,234.33,736.0,2015-04-09 15:00:00,38.00,107.338920,-24.307735,2.249427,-33.681353,51.599259
1,09-04-2015 16.00,34.50,60.25,21.94,16.60,70.19,10.91,0.32,111.72,29.74,...,257.25,72.66,736.0,2015-04-09 16:00:00,34.50,107.339751,-30.533147,2.319932,-33.691369,45.435166
2,09-04-2015 17.00,15.67,52.67,21.78,22.55,58.70,9.66,0.19,112.68,28.70,...,246.50,46.50,736.0,2015-04-09 17:00:00,15.67,107.340583,-33.809317,2.379484,-33.701312,42.209438
3,09-04-2015 18.00,11.33,44.83,16.53,40.28,54.66,12.31,0.40,94.91,26.59,...,242.25,13.92,736.0,2015-04-09 18:00:00,11.33,107.341414,-31.785029,2.427553,-33.711178,44.272759
4,09-04-2015 19.00,23.50,69.33,21.31,60.66,57.12,16.95,0.43,43.89,23.80,...,216.84,5.00,736.0,2015-04-09 19:00:00,23.50,107.342245,-23.351911,2.463718,-33.720968,52.733084


In [88]:
#time features added
df['hour'] = df['ds'].dt.hour
df['dayofweek'] = df['ds'].dt.dayofweek
df['month'] = df['ds'].dt.month
df['dayofyear'] = df['ds'].dt.dayofyear


In [89]:
df.head()

,Timestamp,PM2.5,PM10,NO,NO2,NH3,SO2,CO,Ozone,AT,...,y,trend,daily,weekly,yearly,prophet_yhat,hour,dayofweek,month,dayofyear
0,09-04-2015 15.00,38.00,61.00,NaN,NaN,NaN,8.17,1.08,112.91,30.22,...,38.00,107.338920,-24.307735,2.249427,-33.681353,51.599259,15,3,4,99
1,09-04-2015 16.00,34.50,60.25,21.94,16.60,70.19,10.91,0.32,111.72,29.74,...,34.50,107.339751,-30.533147,2.319932,-33.691369,45.435166,16,3,4,99
2,09-04-2015 17.00,15.67,52.67,21.78,22.55,58.70,9.66,0.19,112.68,28.70,...,15.67,107.340583,-33.809317,2.379484,-33.701312,42.209438,17,3,4,99
3,09-04-2015 18.00,11.33,44.83,16.53,40.28,54.66,12.31,0.40,94.91,26.59,...,11.33,107.341414,-31.785029,2.427553,-33.711178,44.272759,18,3,4,99
4,09-04-2015 19.00,23.50,69.33,21.31,60.66,57.12,16.95,0.43,43.89,23.80,...,23.50,107.342245,-23.351911,2.463718,-33.720968,52.733084,19,3,4,99


In [90]:
X = df[
    base_features +
    ['trend', 'daily', 'weekly', 'yearly', 'prophet_yhat',
     'hour', 'dayofweek', 'month', 'dayofyear']
]

y = df['y']


In [91]:
X.head()

,PM10,NO,NO2,NH3,SO2,CO,Ozone,AT,RH,WS,...,BP,trend,daily,weekly,yearly,prophet_yhat,hour,dayofweek,month,dayofyear
0,61.00,NaN,NaN,NaN,8.17,1.08,112.91,30.22,25.25,3.02,...,736.0,107.338920,-24.307735,2.249427,-33.681353,51.599259,15,3,4,99
1,60.25,21.94,16.60,70.19,10.91,0.32,111.72,29.74,25.75,2.60,...,736.0,107.339751,-30.533147,2.319932,-33.691369,45.435166,16,3,4,99
2,52.67,21.78,22.55,58.70,9.66,0.19,112.68,28.70,27.17,2.41,...,736.0,107.340583,-33.809317,2.379484,-33.701312,42.209438,17,3,4,99
3,44.83,16.53,40.28,54.66,12.31,0.40,94.91,26.59,31.42,1.52,...,736.0,107.341414,-31.785029,2.427553,-33.711178,44.272759,18,3,4,99
4,69.33,21.31,60.66,57.12,16.95,0.43,43.89,23.80,39.83,0.57,...,736.0,107.342245,-23.351911,2.463718,-33.720968,52.733084,19,3,4,99


In [92]:
y.head()

0    38.00
1    34.50
2    15.67
3    11.33
4    23.50
Name: y, dtype: float64

In [93]:
import lightgbm as lgb
from sklearn.model_selection import TimeSeriesSplit

tscv = TimeSeriesSplit(n_splits=5)

model = lgb.LGBMRegressor(
    n_estimators=600,
    learning_rate=0.05,
    max_depth=7,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

for train_idx, val_idx in tscv.split(X):
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        eval_metric='rmse',
    )


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002999 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4652
[LightGBM] [Info] Number of data points in the train set: 12833, number of used features: 22
[LightGBM] [Info] Start training from score 110.346154
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gai

In [94]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

y_pred = model.predict(X)

mae = mean_absolute_error(y, y_pred)
rmse = mean_squared_error(y, y_pred)
r2 = r2_score(y,y_pred)

print("MAE:", mae)
print("RMSE:", rmse)
print("r2:", r2)


MAE: 13.279875505786169
RMSE: 408.90762544451366
r2: 0.9521651819247956
